### --- Day 5: Print Queue ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/5

In [2]:
#!import ../Utils.ipynb

In [3]:
using SCG = System.Collections.Generic;

In [4]:
var inputLines = LoadPuzzleInput(2024, 5);
WriteLines(inputLines);

Loading puzzle file: Day5.txt
Total lines: 1379
Max line length: 68

61|98
25|32
25|15
98|88
98|77


In [5]:
string[] testInputLines = [
    "47|53",
    "97|13",
    "97|61",
    "97|47",
    "75|29",
    "61|13",
    "75|53",
    "29|13",
    "97|29",
    "53|29",
    "61|53",
    "97|53",
    "61|29",
    "47|13",
    "75|47",
    "97|75",
    "47|61",
    "75|61",
    "47|29",
    "75|13",
    "53|13",
    "",
    "75,47,61,53,29",
    "97,61,53,29,13",
    "75,29,13",
    "75,97,47,61,53",
    "61,13,29",
    "97,13,75,29,47",
];

In [6]:
using Rule = (string before, string after);
using PositionDict = SCG.Dictionary<string, int>;

int FilterOrderedPages(string[] inputLines)
{
    var (rulesSection, pageNumbersSection) = inputLines.SeparateBy(str => str is "").ToArray();

    var rulesQ = from rule in rulesSection
                 let ruleBits = rule.Split('|')
                 let before = ruleBits[0]
                 let after = ruleBits[1]
                 select (before, after);
    var rules = rulesQ.ToList();

    var pageNumbersLines = pageNumbersSection.Select(line => line.Split(','));

    var matchLinesQ = from pageNumbers in pageNumbersLines
                      where InOrder(rules, pageNumbers)
                      select GetMiddleNumber(pageNumbers);

    return matchLinesQ.Sum();
}

bool InOrder(List<Rule> rules, string[] pageNumbers)
{
    var positions = pageNumbers.Index().ToDictionary(kv => kv.Item, kv => kv.Index);
    return rules.All(rule => RuleMatches(positions, rule));
}

bool RuleMatches(PositionDict positions, Rule rule) => rule switch
{
    (var before, _) when !positions.ContainsKey(before) => true,
    (_, var after) when !positions.ContainsKey(after) => true,
    _ => positions[rule.before] < positions[rule.after]
};

int GetMiddleNumber(IList<string> pageNumbers) => pageNumbers.Count switch
{
    var length when length % 2 == 0 => throw new ArgumentException("Expected odd length", nameof(pageNumbers)),
    var length => int.Parse(pageNumbers[length / 2])
};

In [7]:
// These have middle page numbers of 61, 53, and 29 respectively. Adding these
// page numbers together gives 143.

var testAnswer = FilterOrderedPages(testInputLines);
Console.WriteLine(testAnswer);

143


In [8]:
var part1Answer = FilterOrderedPages(inputLines);
Console.WriteLine(part1Answer);

6384


In [9]:
// 6384 is correct!
Ensure(6384, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/5

In [11]:
// To solve this, I think we can use the rules to define a custom sort ordering,
// and then sort each list.

using RuleDict = SCG.Dictionary<string, SCG.HashSet<string>>;

class RuleComparer(RuleDict ruleDict) : IComparer<string>
{
    RuleDict _ruleDict = ruleDict;
    
    public int Compare(string x, string y)
    {
        var hasX = _ruleDict.TryGetValue(x, out var xPos);
        var hasY = _ruleDict.TryGetValue(y, out var yPos);

        return (hasX, hasY) switch {
            (true, _) when xPos.Contains(y) => -1, // x before y
            (_, true) when yPos.Contains(x) => 1, // y before x
            _ => 0 // no change
        };
    }
}

In [12]:
int FilterUnorderedPages(string[] inputLines, int part1Answer)
{
    var (rulesSection, pageNumbersSection) = inputLines.SeparateBy(str => str is "").ToArray();

    var pageNumbersLines = pageNumbersSection.Select(line => line.Split(',').ToList());

    RuleDict ruleDict = new();
    foreach (var rule in rulesSection)
    {
        var (before, after) = rule.Split('|');

        var afters = ruleDict.GetValueOrDefault(before, new());
        afters.Add(after);
        ruleDict[before] = afters;
    }
    RuleComparer ruleComparer = new(ruleDict);

    var sum = 0;
    foreach (var line in pageNumbersLines)
    {
        line.Sort(ruleComparer);
        sum += GetMiddleNumber(line);
    }

    return sum - part1Answer;
}

In [13]:
// After taking only the incorrectly-ordered updates and ordering them
// correctly, their middle page numbers are 47, 29, and 47. Adding these together
// produces 123.

var part2TestAnswer = FilterUnorderedPages(testInputLines, testAnswer);
Console.WriteLine(part2TestAnswer);

123


In [14]:
// Find the updates which are not in the correct order. What do you get if you
// add up the middle page numbers after correctly ordering just those updates?

var part2Answer = FilterUnorderedPages(inputLines, part1Answer);
Console.WriteLine(part2Answer);

5353


In [15]:
// 5353 is correct!
Ensure(5353, part2Answer);